In [1]:
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import PyPDF2

In [3]:
def getTextPDF(pdfFileName, Password = ''):
    pdf_file = open(pdfFileName, 'rb')
    read_pdf = PdfFileReader(pdf_file)
    if Password !='':
        read_pdf.decrypt(Password)
    text = []
    for i in range(0, read_pdf.getNumPages()):
        text.append(read_pdf.getPage(i).extractText())
        
    return '\n'.join(text)

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(file):
    
    rsrcmgr = PDFResourceManager()
    
    retstr = StringIO()
    
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams = laparams)
    fp = open(file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching=caching, check_extractable = True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
convert_pdf_to_txt('15_9.pdf')

'금융통화위원회 의사록\n\n2011년도 제 차 회의\n\n29\n\n1.\n\n일\n\n자\n\n2011\n\n년 월 일 금\n)\n\n12\n\n22\n\n(\n\n2.\n\n장\n\n소\n\n금융통화위원회 회의실\n\n3.\n\n출석위원\n\n김 대 식 위 원 의장직무대행\n)\n\n(\n\n최 도 성 위 원\n\n강 명 헌 위 원\n\n이 주 열 위 원 부총재\n)\n\n(\n\n임 승 태 위 원\n\n4.\n\n결석위원\n\n김 중 수 의 장 총재(\n)\n\n5.\n\n참 여 자\n\n강 태 혁 감 사\n\n장 병 화 부총재보\n\n장 세 근 부총재보\n\n김 재 천 부총재보\n\n이 광 준 부총재보\n\n박 원 식 부총재보\n\n추 흥 식 외자운용원장\n\n정 희 전 정책기획국장\n\n민 성 기 금융시장국장\n\n김 종 화 국제국장\n\n김 윤 철 금융통화위원회실장 이 용 회 공보실장\n\n성 상 경 의사관리팀장\n\n6.\n\n회의경과\n\n가 의결안건\n\n.\n\n<\n\n의안 제 호\n\n63\n\n2012\n\n년도 분기 한국은행 총액한도대출의 한도 결정\n\n1/4\n\n>\n\n(\n\n)\n\n의장직무대행이 한국은행법 제 조 및 한국은행의 금융기관대출규정\n\n28\n\n제 조에9\n\n의거하여 의안 제 호\n\n63\n\n도 결정 을 상정하였음\n\n(\n\n)\n\n위원 토의내용\n\n2012\n\n년도 분기 한국은행 총액한도대출의 한\n\n1/4\n\n- 1 -\n\n\x0c일부 위원은 당행은 그동안 금융시장 여건 개선추세를 보아가며 글로벌\n\n금융위기 대응과정에서 확대하였던 총액대출한도를 전면적으로 축소하여 왔다고\n\n언급한 후 최근 은행의 중소기업대출 추이를 볼 때 신용공급이 크게 위축되는\n\n,\n\n모습은 나타나고 있지 않지만 국내외 경제여건의 불확실성 증대 등에 따라 은행\n\n,\n\n들의 대출태도가 강화되고 일부 업황 부진업종을 중심으로 자금사정이 악화되는\n\n움직임을 보이고 있는 점 앞으로도 경기의 완만한 

In [6]:
from urllib import request
from PyPDF2 import pdf
from urllib.request import urlopen

In [4]:
import os
import pandas as pd
# result_list = []
for i in range(1, 45):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.content)
    li_list = []
    li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
#     print(len(li_list))
#     print(li_list)
    print(i)
    result_list = []
    for x in range(0, len(li_list)):
        new_dict = {}
        link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
        for link in link_li:
            if link.find('a').attrs['title'][-3:] == 'pdf':
                link_u = link.find('a').attrs['href']
                title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                url2 = 'http://www.bok.or.kr' + link_u
                file_res2 = requests.get(url2)
                file_name = '{}.pdf'.format(title)
                with open(file_name, 'wb') as f:
                    f.write(file_res2.content)
                try:
                    text = convert_pdf_to_txt(file_name)
                    new_dict = {
                        'title' : title,
                        'text' : text
                    }
                    result_list.append(new_dict)

                except:
                    print('빠진 목록 : {}'.format(title))
                    
                              
    df = pd.DataFrame(result_list)
    BASE_DIR = os.getcwd()
    SAVE_DB_DIR = os.path.join(BASE_DIR, '금통위')

    if not os.path.exists(SAVE_DB_DIR):
        os.makedirs(SAVE_DB_DIR)

    #텍스트 파일 만들기
    temp_name = f"{title}.xlsx"
    txt_name = os.path.join(SAVE_DB_DIR, temp_name)
    excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')

    df.to_excel(excel_writer)

    excel_writer.save()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


AttributeError: 'NoneType' object has no attribute 'find'

In [4]:
import os
import pandas as pd

for i in range(1, 45):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.content)
    li_list = []
    li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
#     print(len(li_list))
#     print(li_list)
    print(i)
    for x in range(0, len(li_list)):

        try:
            new_dict = {}
            link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
            for link in link_li:
                if link.find('a').attrs['title'][-3:] == 'pdf':
                    link_u = link.find('a').attrs['href']
                    title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                    url2 = 'http://www.bok.or.kr' + link_u
                    file_res2 = requests.get(url2)
                    file_name = '{}.pdf'.format(title)
                    with open(file_name, 'wb') as f:
                        f.write(file_res2.content)
                    try:
                        text = getTextPDF(file_name)
                        new_dict = {
                            'title' : title,
                            'text' : text
                        }
                        df = pd.DataFrame(new_dict)
                        BASE_DIR = os.getcwd()
                        SAVE_DB_DIR = os.path.join(BASE_DIR, '금통위')

                        if not os.path.exists(SAVE_DB_DIR):
                            os.makedirs(SAVE_DB_DIR)

                        #텍스트 파일 만들기
                        temp_name = f"{title}.xlsx"
                        txt_name = os.path.join(SAVE_DB_DIR, temp_name)
                        excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
                
                        df.to_excel(excel_writer)
                        
                        excel_writer.save()
                    except:
                        print('빠진 목록 : {}'.format(title))
        except:
            link2 = li_list[x].find('div').find('div').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url3 = 'http://www.bok.or.kr' + link2
            file_res3 = requests.get(url3)
            
            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res3.content)

1
빠진 목록 : 금융통화위원회 의사록(2019년도 제12차)(2019.6.20.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제10차)(2019.5.31.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제9차)(2019.5.9)
빠진 목록 : 금융통화위원회 의사록(2019년도 제7차)(2019.4.18.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제6차)(2019.3.28.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제4차)(2019.2.28.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제3차)(2019.2.14.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제2차)(2019.1.24.)
빠진 목록 : 금융통화위원회 의사록(2019년도 제1차)(2019.1.17.)
빠진 목록 : 금융통화위원회 의사록(2018년도 제25차)(2018.12.26.) 
2
빠진 목록 : 금융통화위원회 의사록(2018년도 제24차)(2018.12.20.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제23차)(2018.12.6.)
빠진 목록 : 금융통화위원회 의사록(2018년도 제22차)(2018.11.30.)
빠진 목록 : 금융통화위원회 의사록(2018년도 제21차)(2018.11.8.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제19차)(2018.10.18.)
빠진 목록 : 금융통화위원회 의사록(2018년도 제18차)(2018.9.20.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제16차)(2018.8.31.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제14차)(2018.7.26.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제13차)(2018.7.12.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제12차)(2018.6.20.) 
3
빠진 목록 : 금융통화위원회 의사록(2018년도 제10차)(2018.5.24.) 
빠진 목록 : 금융통화위원회 의사록(2018년도 제8차)(2018.4.26.)
빠

빠진 목록 : 금융통화위원회 의사록(2010년도 제8차)(2010.4.9)
빠진 목록 : 금융통화위원회 의사록(2010년도 제7차)(2010.3.25)
빠진 목록 : 금융통화위원회 의사록(2010년도 제6차)(2010.3.11)
빠진 목록 : 금융통화위원회 의사록(2010년도 제4차)(2010.2.11)
빠진 목록 : 금융통화위원회 의사록(2010년도 제3차)(2010.1.21)
20
빠진 목록 : 금융통화위원회 의사록(2010년도 제1차)(2010.1.7)
빠진 목록 : 금융통화위원회 의사록(2010년도 제2차)(2010.1.8)
빠진 목록 : 금융통화위원회 의사록(2009년도 제27차)(2009.12.24)
빠진 목록 : 금융통화위원회 의사록(2009년도 제26차)(2009.12.10)
빠진 목록 : 금융통화위원회 의사록(2009년도 제25차)(2009.11.26)
빠진 목록 : 금융통화위원회 의사록(2009년도 제24차)(2009.11.12)
빠진 목록 : 금융통화위원회 의사록(2009년도 제22차)(2009.10.9)
빠진 목록 : 금융통화위원회 의사록(2009년도 제21차)(2009.9.24)
빠진 목록 : 금융통화위원회 의사록(2009년도 제20차)(2009.9.10)
빠진 목록 : 금융통화위원회 의사록(2009년도 제18차)(2009.8.11)
21
빠진 목록 : 금융통화위원회 의사록(2009년도 제17차)(2009.7.23)
빠진 목록 : 금융통화위원회 의사록(2009년도 제16차)(2009.7.9)
빠진 목록 : 금융통화위원회 의사록(2009년도 제15차)(2009.6.25)
빠진 목록 : 금융통화위원회 의사록(2009년도 제14차)(2009.6.11)
빠진 목록 : 금융통화위원회 의사록(2009년도 제13차)(2009.5.21)
빠진 목록 : 금융통화위원회 의사록(2009년도 제12차)(2009.5.12)
빠진 목록 : 금융통화위원회 의사록(2009년도 제11차)(2009.4.30)
빠진 목록 : 금융통화위원회 의사록(2009년도 제10차)(

빠진 목록 : 금융통화위원회 의사록(2008년도 제2차)(2008.1.10)
빠진 목록 : 금융통화위원회 의사록(2008년도 제1차)(2008.1.4)
빠진 목록 : 금융통화위원회 의사록(2007년도 제26차)(2007.12.20)
25
빠진 목록 : 금융통화위원회 의사록(2007년도 제25차)(2007.12.7)
빠진 목록 : 금융통화위원회 의사록(2007년도 제24차)(2007.11.22)
빠진 목록 : 금융통화위원회 의사록(2007년도 제23차)(2007.11.8)
빠진 목록 : 금융통화위원회 의사록(2007년도 제21차)(2007.10.11)
빠진 목록 : 금융통화위원회 의사록(2007년도 제20차)(2007.9.20)
빠진 목록 : 금융통화위원회 의사록(2007년도 제19차)(2007.9.7)
빠진 목록 : 금융통화위원회 의사록(2007년도 제17차)(2007.8.9)
빠진 목록 : 금융통화위원회 의사록(2007년도 제16차)(2007.7.26)
빠진 목록 : 금융통화위원회 의사록(2007년도 제15차)(2007.7.12)
빠진 목록 : 금융통화위원회 의사록(2007년도 제14차)(2007.6.21)
26
빠진 목록 : 금융통화위원회 의사록(2007년도 제13차)(2007.6.8)
빠진 목록 : 금융통화위원회 의사록(2007년도 제11차)(2007.5.10)
빠진 목록 : 금융통화위원회 의사록(2007년도 제9차)(2007.4.12)
빠진 목록 : 금융통화위원회 의사록(2007년도 제8차)(2007.3.22)
빠진 목록 : 금융통화위원회 의사록(2007년도 제7차)(2007.3.8)
빠진 목록 : 금융통화위원회 의사록(2007년도 제4차)(2007.2.8)
빠진 목록 : 금융통화위원회 의사록(2007년도 제3차)(2007.1.25)
빠진 목록 : 금융통화위원회 의사록(2007년도 제2차)(2007.1.11)
빠진 목록 : 금융통화위원회 의사록(2007년도 제1차)(2007.1.5)
빠진 목록 : 금융통화위원회 의사록(2006년도 제26차)(2006.1

In [ ]:
convert_pdf_to_txt(file):

In [ ]:
b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
params = {
'pageIndex' : 1
}
resp = requests.get(b_url, params = params)
soup = BeautifulSoup(resp.content)
li_list = []
li= soup.find('div', class_='bdLine type2').find('ul').find('li')

In [ ]:
li.find('div', class_='fileGoupBox').find('ul').find_all('li')[0].find('a').attrs['title'][-3:]

In [ ]:
from urllib import request
import shutil

In [ ]:
c_url = 'http://www.bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000012003&fileSn=1'
a = urlopen(c_url)

In [ ]:
resp_c = requests.get(c_url)

In [ ]:
soup = BeautifulSoup(resp_c.content)

In [ ]:
a = soup.find('div', class_='fileGoupBox')
a

In [ ]:
link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')

In [ ]:
with open('34.pdf', 'wb') as f:
    resp_c.raw.decode_content = True
    shutil.copyfileobj(resp_c.raw, f)

In [ ]:
import unicode

In [ ]:
data = request.urlopen(c_url).read()
# date2unicode(euc_bytes,'euc-kr').encode('utf-8')

text = data.decode('euc-kr')

text

In [ ]:
a

In [ ]:
 convert_pdf_to_txt(a)

In [ ]:
b = request.urlretrieve(c_url)
b

In [ ]:
 convert_pdf_to_txt(b)

In [ ]:
help(resp.raw)

In [19]:
b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
params = {
'pageIndex' : 15
}
resp = requests.get(b_url, params = params)
soup = BeautifulSoup(resp.content)
li_list = []
li= soup.find('div', class_='bdLine type2').find('ul').find('li')
li_li = li.find_next_siblings('li')
li_list.append(li)
li_list.extend(li_li)

#     print(len(li_list))
#     print(li_list)
result_list = []
for x in range(0, len(li_list)):
    new_dict = {}
    link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')
    for link in link_li:
        if link.find('a').attrs['title'][-3:] == 'pdf':
            link_u = link.find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url2 = 'http://www.bok.or.kr' + link_u
            file_res2 = requests.get(url2)
            file_name = '{}.pdf'.format(title)
            with open(file_name, 'wb') as f:
                f.write(file_res2.content)

            text = convert_pdf_to_txt(file_name)
            new_dict = {
                'title' : title,
                'text' : text
            }
            result_list.append(new_dict)


AttributeError: 'PDFStream' object has no attribute 'replace'